In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install streamlit ollama firebase-admin

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
--2025-11-07 13:04:38--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.0/cloudflared-linux-amd64.deb [following]
--2025-11-07 13:04:39--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.0/cloudflared-linux-amd64.deb
Reusing existing connectio

In [ ]:
import subprocess, threading, os, time
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"
subprocess.Popen(["ollama", "serve"], env=env)
time.sleep(10)
print("Ollama Server running...")

Ollama Server running...


In [ ]:
!ollama pull mistral

import subprocess
import time
import re

print("Đang tạo Cloudflare Tunnel cho Ollama...")
tunnel_process = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:11434"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)
time.sleep(5)
ollama_public_url = None

for line in tunnel_process.stderr:
    print(line.strip())
    if "trycloudflare.com" in line:
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if match:
            ollama_public_url = match.group(0)
            print(f"\nOllama Public URL: {ollama_public_url}")
            break
    time.sleep(0.1)

if not ollama_public_url:
    print("Không tìm thấy URL. Hãy kiểm tra logs phía trên.")
else:
    with open('ollama_url.txt', 'w') as f:
        f.write(ollama_public_url)
    print(f"\n📝 Đã lưu URL vào file ollama_url.txt")


Đang tạo Cloudflare Tunnel cho Ollama...
2025-11-07T13:05:07Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-07T13:05:07Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-07T13:05:12Z INF +--------------------------------------------------------------------------------------------+
2025-11-07T13:05:12Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-07T13:05:12Z INF |  https://ata

In [ ]:
%%writefile app.py
# -*- coding: utf-8 -*-
import streamlit as st
import requests
import firebase_admin
from firebase_admin import credentials, auth, firestore

# Khởi tạo
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False

def init_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate("chatbot_tour_key.json")
        firebase_admin.initialize_app(cred)
        st.session_state.db = firestore.client()

def login(email, password, is_signup=False):
    init_firebase()
    try:
        if is_signup:
            user = auth.create_user(email=email, password=password)
            st.success("Đăng ký thành công!")
        else:
            user = auth.get_user_by_email(email)
            st.session_state.user_id = user.uid
            st.session_state.logged_in = True
            st.rerun()
    except Exception as e:
        st.error(f"Lỗi: {str(e)[:50]}")

def generate_plan(origin, dest, dates, interests, pace, ollama_url):
    prompt = f"""Tạo lịch trình du lịch từ {origin} đến {dest} trong {dates}.

YÊU CẦU:
- Tiếng Việt có dấu
- Tốc độ: {pace}
- Sở thích: {interests}
- Chia theo ngày, mỗi ngày có: Sáng (Morning), Chiều (Afternoon), Tối (Evening)
- Mỗi địa điểm có giờ cụ thể (8h00, 14h30...) và giải thích ngắn

Ví dụ:
Ngày 1:
Sáng:
- 7h00: Khởi hành từ {origin}
- 10h00: Đến {dest}

Chiều:
- 13h00: Ăn trưa tại ABC (đặc sản: XYZ)
- 14h30: Tham quan DEF

Tối:
- 18h00: Ăn tối
- 20h00: Dạo phố"""

    if not ollama_url or not ollama_url.startswith("http"):
        return "❌ Lỗi: Ollama URL không hợp lệ"

    try:
        res = requests.post(
            f"{ollama_url}/api/generate",
            json={"model": "mistral", "prompt": prompt, "stream": False},
            timeout=180,
            headers={'Content-Type': 'application/json'}
        )

        if res.status_code == 200:
            return res.json()['response']
        else:
            return f"❌ Lỗi API (Code {res.status_code})"

    except requests.exceptions.Timeout:
        return "⏱️ Timeout! Thử giảm số ngày hoặc chạy lại."

    except requests.exceptions.ConnectionError:
        return "❌ Không kết nối được! Chạy lại Cell 3 để tạo URL mới."

    except Exception as e:
        return f"❌ Lỗi: {str(e)[:100]}"

# UI
st.title("🗺️ Travel Planner")

# Login
if not st.session_state.logged_in:
    email = st.text_input("Email")
    password = st.text_input("Password", type="password")

    col1, col2 = st.columns(2)
    if col1.button("Login"):
        login(email, password)
    if col2.button("Sign Up"):
        login(email, password, True)

# Main App
else:
    init_firebase()

    # Logout
    if st.button("Logout"):
        st.session_state.logged_in = False
        st.rerun()

    # Get Ollama URL
    try:
        with open('ollama_url.txt', 'r', encoding='utf-8') as f:
            ollama_url = f.read().strip()
    except:
        ollama_url = ""

    ollama_url = st.text_input("Ollama URL:", value=ollama_url)

    if ollama_url:
        st.divider()

        # Input Form
        origin = st.text_input("Origin City:", "Hà Nội")
        dest = st.text_input("Destination City:", "Đà Nẵng")
        dates = st.text_input("Dates:", "3 days")

        interests = st.multiselect("Interests:",
                                   ["Food", "Museums", "Nature", "Nightlife"],
                                   ["Food"])

        pace = st.radio("Pace:", ["Relaxed", "Normal", "Tight"])

        # Generate
        if st.button("Generate"):
            if not dest.strip():
                st.error("Vui lòng nhập Destination!")
            else:
                interests_str = ", ".join(interests) if interests else "General"

                with st.spinner("Creating..."):
                    result = generate_plan(origin, dest, dates, interests_str, pace, ollama_url)

                st.markdown(result)

                # Save
                if not result.startswith("❌") and not result.startswith("⏱️"):
                    try:
                        st.session_state.db.collection('trips').add({
                            'user': st.session_state.user_id,
                            'query': f"{origin} → {dest} | {dates} | {interests_str} | {pace}",
                            'plan': result,
                            'timestamp': firestore.SERVER_TIMESTAMP
                        })
                    except:
                        pass

        # History
        st.divider()
        st.subheader("History")

        try:
            trips = (st.session_state.db.collection('trips')
                    .where('user', '==', st.session_state.user_id)
                    .order_by('timestamp', direction=firestore.Query.DESCENDING)
                    .limit(5)
                    .stream())

            for trip in trips:
                data = trip.to_dict()
                with st.expander(data.get('query', 'N/A')):
                    st.write(data.get('plan', ''))
        except:
            pass

Overwriting app.py


In [ ]:
import subprocess
import time
import re
import threading

# Khởi chạy Streamlit ở background
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port=8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("⏳ Đang khởi động Streamlit...")
time.sleep(5)

# Tạo Cloudflare Tunnel cho Streamlit
print("\n🚀 Đang tạo Cloudflare Tunnel cho Streamlit...")
tunnel_process = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Lấy URL từ output
streamlit_public_url = None
for line in tunnel_process.stderr:
    print(line.strip())
    if "trycloudflare.com" in line:
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if match:
            streamlit_public_url = match.group(0)
            print(f"\n✅ Streamlit Public URL: {streamlit_public_url}")
            print(f"\n🌐 Truy cập ứng dụng tại: {streamlit_public_url}")
            break

if not streamlit_public_url:
    print("⚠️ Không tìm thấy URL. Kiểm tra logs phía trên.")

print("\n📌 Lưu ý: Giữ cell này chạy để tunnel hoạt động!")

⏳ Đang khởi động Streamlit...

🚀 Đang tạo Cloudflare Tunnel cho Streamlit...
2025-11-07T13:05:36Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-07T13:05:36Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-07T13:05:39Z INF +--------------------------------------------------------------------------------------------+
2025-11-07T13:05:39Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025